In [97]:
import pandas as pd
from padelpy import from_smiles
from rdkit.Chem import AllChem as Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PandasTools
from rdkit.Chem import Draw, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit import DataStructs
from padelpy import from_smiles, from_mdl, from_sdf, padeldescriptor

import sirms as si

import numpy as np
import seaborn as sns
import matplotlib

In [98]:
datafile = pd.read_csv("../1-dados/DADOS/SMILES/COVID-Sulkowsk-LabTable.csv", delimiter= ",")

In [99]:
PandasTools.AddMoleculeColumnToFrame(datafile,'SMILES','Molecule',includeFingerprints=True)
>>> print([str(x) for x in  datafile.columns])

['ID', 'SMILES', 'activity', 'Molecule']


# Gerando descritores PADEL

In [100]:
padeldescriptor(mol_dir="../1-dados/DADOS/SDF/COVID-Sulkowsk.sdf", d_file='./DESCRITORES/PADEL/padel_descriptors.csv', d_2d=True, d_3d=True)

# Gerando descritores RDKIT

In [101]:
mols = [Chem.MolFromSmiles(smi) for smi in datafile['SMILES'].array.shift().dropna()]
rdkit_2d_desc = []
calc = MoleculeDescriptors.MolecularDescriptorCalculator([x[0] for x in Descriptors._descList])
header = calc.GetDescriptorNames()
for i in range(len(mols)):
    ds = calc.CalcDescriptors(mols[i])
    rdkit_2d_desc.append(ds)
df = pd.DataFrame(rdkit_2d_desc,columns=header)
df.to_csv('./DESCRITORES/RDKIT/rdkit_Molecule_descriptors.csv')

# Gerando descritores MorganFingerprint

In [102]:
FP = [Chem.GetMorganFingerprintAsBitVect(m, 2, 1024) for m in datafile['Molecule'].array.shift().dropna()]
df_FP = pd.DataFrame(np.array(FP))
df.to_csv('./DESCRITORES/MORGAN/morgan_fingerprint_descriptors.csv')